In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date



dfm = pd.read_excel('workout.xlsx')

for col in ["Begin", "End"]:
    timestr = [ 0 for i in range(dfm.shape[0])]
    for i in range(dfm.shape[0]):
        timestr[i] = dfm[col][i].strftime('%H:%M:%S')
    dfm[col] = pd.to_timedelta(timestr)

dfm["Duration"] = (dfm["End"] - dfm["Begin"]).dt.seconds//60

labels = {"Date": "Дата", 
          "Duration": "Длительность, мин",
          "Weight":"Вес, кг"}

In [8]:
import plotly.express as px
import plotly
fig = px.area(dfm, y="Duration", x="Date", labels=labels)
fig.show()
plotly.offline.iplot(fig, filename='duration.png')

In [9]:
fig = px.area(dfm, y="Weight", x="Date", labels=labels)
fig.update_yaxes(range=[dfm["Weight"].min(), dfm["Weight"].max()])
plotly.offline.iplot(fig, filename='myweight.png')

# Оценка максимального веса

Взял из [Википедии](https://ru.wikipedia.org/wiki/%D0%9E%D0%B4%D0%BD%D0%BE_%D0%BF%D0%BE%D0%B2%D1%82%D0%BE%D1%80%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81_%D0%BC%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%BC_%D0%B2%D0%B5%D1%81%D0%BE%D0%BC#%D0%9E'%D0%9A%D0%BE%D0%BD%D0%BD%D0%BE%D1%80_(O'Conner)_%D0%B8_%D0%B4%D1%80) формулу О'Коннора и посчитал по ней каждый мой день, когда я занимался жимом.

In [10]:
dates = dfm["Date"]
N = dates.size
f = lambda w,c: w * ( 1 + 0.025*c )

score = np.zeros(N)
for i, _date in enumerate(dates):
    df = pd.read_excel('workout.xlsx', sheet_name=_date)
    try:
        weight = df["Bench press, weight"]
        count = df["Bench press, count"]
        y = np.zeros(count.size)

        for j in range(count.size):
            y[j] = f(weight[j],count[j])
            
        score[i] = y.max()
    except:
        continue
    
fig = px.bar(dfm, y=score, x="Date", labels={"y": "Расчетный разовый максимум", "Date":"Дата"})
fig.show()

plotly.offline.iplot(fig, filename='maxweight.png')

In [11]:
dates = dfm["Date"]
N = dates.size
f = lambda w,c: w * c

score = np.zeros(N)
for i, _date in enumerate(dates):
    df = pd.read_excel('workout.xlsx', sheet_name=_date)
    try:
        weight = df["Bench press, weight"]
        count = df["Bench press, count"]
        y = np.zeros(count.size)

        for j in range(count.size):
            y[j] = f(weight[j],count[j])
            
        score[i] = np.sum(y)
    except:
        continue
    
fig = px.bar(dfm, y=score/1000, x="Date", labels={"y": "Суммарный поднятый вес, т", "Date":"Дата"})
fig.show()
plotly.offline.iplot(fig, filename='sumweight.png')